In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import accuracy_score
import random
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import callbacks
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from numpy.random import seed
import pandas as pd
import matplotlib as mat
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import glob
import cv2
from tensorflow.random import set_seed
import warnings
%matplotlib inline

In [3]:
pd.options.display.max_colwidth = 100
seed(42)
random.seed(42)
os.environ['PYTHONHASHSEED'] = str(42)
os.environ['TF_DETERMINISTIC_OPS'] = '1'
set_seed(42)
warnings.filterwarnings('ignore')

In [4]:
IMG_SIZE = 224
BATCH = 32
SEED = 42

In [5]:
main_path = "/content/drive/MyDrive/chest_xray"

train_path = os.path.join(main_path,"train")

test_path=os.path.join(main_path,"test")

train_normal = glob.glob(train_path+"/NORMAL/*.jpeg")

train_pneumonia = glob.glob(train_path+"/PNEUMONIA/*.jpeg")

test_normal = glob.glob(test_path+"/NORMAL/*.jpeg")

test_pneumonia = glob.glob(test_path+"/PNEUMONIA/*.jpeg")

In [6]:
train_list = [x for x in train_normal]
train_list.extend([x for x in train_pneumonia])

df_train = pd.DataFrame(np.concatenate([['Normal']*len(train_normal) , ['Pneumonia']*len(train_pneumonia)]), columns = ['class'])
df_train['image'] = [x for x in train_list]

test_list = [x for x in test_normal]
test_list.extend([x for x in test_pneumonia])

df_test = pd.DataFrame(np.concatenate([['Normal']*len(test_normal) , ['Pneumonia']*len(test_pneumonia)]), columns = ['class'])
df_test['image'] = [x for x in test_list]

In [7]:
df_train

,class,image
0,Normal,/content/drive/MyDrive/chest_xray/train/NORMAL/NORMAL-3342765-0001.jpeg
1,Normal,/content/drive/MyDrive/chest_xray/train/NORMAL/NORMAL-3269314-0001.jpeg
2,Normal,/content/drive/MyDrive/chest_xray/train/NORMAL/NORMAL-3342765-0002.jpeg
3,Normal,/content/drive/MyDrive/chest_xray/train/NORMAL/NORMAL-3250852-0001.jpeg
4,Normal,/content/drive/MyDrive/chest_xray/train/NORMAL/NORMAL-3526928-0001.jpeg
...,...,...
5227,Pneumonia,/content/drive/MyDrive/chest_xray/train/PNEUMONIA/BACTERIA-4057077-0001.jpeg
5228,Pneumonia,/content/drive/MyDrive/chest_xray/train/PNEUMONIA/BACTERIA-3955228-0001.jpeg
5229,Pneumonia,/content/drive/MyDrive/chest_xray/train/PNEUMONIA/BACTERIA-4371619-0002.jpeg
5230,Pneumonia,/content/drive/MyDrive/chest_xray/train/PNEUMONIA/BACTERIA-387616-0001.jpeg


In [8]:
df_test

,class,image
0,Normal,/content/drive/MyDrive/chest_xray/test/NORMAL/NORMAL-1160949-0001.jpeg
1,Normal,/content/drive/MyDrive/chest_xray/test/NORMAL/NORMAL-115218-0001.jpeg
2,Normal,/content/drive/MyDrive/chest_xray/test/NORMAL/NORMAL-11419-0001.jpeg
3,Normal,/content/drive/MyDrive/chest_xray/test/NORMAL/NORMAL-1049278-0001.jpeg
4,Normal,/content/drive/MyDrive/chest_xray/test/NORMAL/NORMAL-1212407-0001.jpeg
...,...,...
656,Pneumonia,/content/drive/MyDrive/chest_xray/test/PNEUMONIA/VIRUS-9476607-0001.jpeg
657,Pneumonia,/content/drive/MyDrive/chest_xray/test/PNEUMONIA/VIRUS-9342284-0001.jpeg
658,Pneumonia,/content/drive/MyDrive/chest_xray/test/PNEUMONIA/VIRUS-8377377-0001.jpeg
659,Pneumonia,/content/drive/MyDrive/chest_xray/test/PNEUMONIA/VIRUS-9564688-0002.jpeg


In [11]:
train_df, val_df = train_test_split(df_train, test_size = 0.20, random_state = SEED, stratify = df_train['class'])

In [12]:
train_df

,class,image
3566,Pneumonia,/content/drive/MyDrive/chest_xray/train/PNEUMONIA/BACTERIA-4799529-0001.jpeg
2866,Pneumonia,/content/drive/MyDrive/chest_xray/train/PNEUMONIA/BACTERIA-9530903-0004.jpeg
2681,Pneumonia,/content/drive/MyDrive/chest_xray/train/PNEUMONIA/BACTERIA-864208-0002.jpeg
1199,Normal,/content/drive/MyDrive/chest_xray/train/NORMAL/NORMAL-2386123-0001.jpeg
4619,Pneumonia,/content/drive/MyDrive/chest_xray/train/PNEUMONIA/BACTERIA-1797366-0001.jpeg
...,...,...
3476,Pneumonia,/content/drive/MyDrive/chest_xray/train/PNEUMONIA/BACTERIA-4932029-0003.jpeg
678,Normal,/content/drive/MyDrive/chest_xray/train/NORMAL/NORMAL-7824011-0001.jpeg
1560,Pneumonia,/content/drive/MyDrive/chest_xray/train/PNEUMONIA/VIRUS-52163-0003.jpeg
2769,Pneumonia,/content/drive/MyDrive/chest_xray/train/PNEUMONIA/BACTERIA-9424776-0001.jpeg


In [13]:
val_df

,class,image
2945,Pneumonia,/content/drive/MyDrive/chest_xray/train/PNEUMONIA/BACTERIA-9835318-0001.jpeg
4878,Pneumonia,/content/drive/MyDrive/chest_xray/train/PNEUMONIA/BACTERIA-3362849-0001.jpeg
3177,Pneumonia,/content/drive/MyDrive/chest_xray/train/PNEUMONIA/VIRUS-2427242-0005.jpeg
972,Normal,/content/drive/MyDrive/chest_xray/train/NORMAL/NORMAL-9756687-0001.jpeg
3059,Pneumonia,/content/drive/MyDrive/chest_xray/train/PNEUMONIA/VIRUS-1591633-0002.jpeg
...,...,...
253,Normal,/content/drive/MyDrive/chest_xray/train/NORMAL/NORMAL-5075834-0001.jpeg
4315,Pneumonia,/content/drive/MyDrive/chest_xray/train/PNEUMONIA/BACTERIA-7847892-0004.jpeg
687,Normal,/content/drive/MyDrive/chest_xray/train/NORMAL/NORMAL-7953615-0001.jpeg
3417,Pneumonia,/content/drive/MyDrive/chest_xray/train/PNEUMONIA/BACTERIA-4352107-0001.jpeg


In [14]:
train_datagen = ImageDataGenerator(rescale=1/255.,
                                  zoom_range = 0.1,

                                  width_shift_range = 0.1,
                                  height_shift_range = 0.1)

val_datagen = ImageDataGenerator(rescale=1/255.)

ds_train = train_datagen.flow_from_dataframe(train_df,

                                             x_col = 'image',
                                             y_col = 'class',
                                             target_size = (IMG_SIZE, IMG_SIZE),
                                             class_mode = 'binary',
                                             batch_size = BATCH,
                                             seed = SEED)

ds_val = val_datagen.flow_from_dataframe(val_df,

                                            x_col = 'image',
                                            y_col = 'class',
                                            target_size = (IMG_SIZE, IMG_SIZE),
                                            class_mode = 'binary',
                                            batch_size = BATCH,
                                            seed = SEED)

ds_test = val_datagen.flow_from_dataframe(df_test,

                                            x_col = 'image',
                                            y_col = 'class',
                                            target_size = (IMG_SIZE, IMG_SIZE),
                                            class_mode = 'binary',
                                            batch_size = 1,
                                            shuffle = False)

Found 4185 validated image filenames belonging to 2 classes.
Found 1047 validated image filenames belonging to 2 classes.
Found 661 validated image filenames belonging to 2 classes.


In [15]:
base_model = tf.keras.applications.ResNet152V2(
    weights='imagenet',
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    include_top=False)

base_model.trainable = False

def get_pretrained():

    inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))

    x = base_model(inputs)

    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.1)(x)


    output = layers.Dense(1, activation='sigmoid')(x)

    model = keras.Model(inputs=[inputs], outputs=output)

    return model


234545216/234545216 [==============================] - 2s 0us/step


In [16]:
keras.backend.clear_session()

model_pretrained = get_pretrained()
model_pretrained.compile(loss='binary_crossentropy'
              , optimizer = keras.optimizers.Adam(learning_rate=5e-5), metrics='binary_accuracy')

In [17]:
early_stopping = callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    min_delta=1e-7,
    restore_best_weights=True,
)

plateau = callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor = 0.2,
    patience = 2,
    min_delt = 1e-7,
    cooldown = 0,
    verbose = 1
)

In [18]:
history = model_pretrained.fit(ds_train,
          batch_size = BATCH, epochs = 4,
          validation_data=ds_val,
          callbacks=[early_stopping, plateau],
          steps_per_epoch=(len(train_df)/BATCH),
          validation_steps=(len(val_df)/BATCH));

Epoch 1/4
130/130 [==============================] - 1892s 14s/step - loss: 0.3432 - binary_accuracy: 0.8595 - val_loss: 0.2035 - val_binary_accuracy: 0.9265 - lr: 5.0000e-05
Epoch 2/4
130/130 [==============================] - 147s 1s/step - loss: 0.1927 - binary_accuracy: 0.9314 - val_loss: 0.1590 - val_binary_accuracy: 0.9446 - lr: 5.0000e-05
Epoch 3/4
130/130 [==============================] - 146s 1s/step - loss: 0.1568 - binary_accuracy: 0.9436 - val_loss: 0.1444 - val_binary_accuracy: 0.9465 - lr: 5.0000e-05
Epoch 4/4
130/130 [==============================] - 147s 1s/step - loss: 0.1391 - binary_accuracy: 0.9486 - val_loss: 0.1334 - val_binary_accuracy: 0.9532 - lr: 5.0000e-05


In [19]:
score = model_pretrained.evaluate(ds_val, steps = len(val_df)/BATCH, verbose = 0)

In [20]:
print('Training Loss for ResNet + Transfer Learning: ',score[0])
print('Training Accuracy for  ResNet + Transfer Learning: ',score[1]*100, '%')

Training Loss for ResNet + Transfer Learning:  0.13335129618644714
Training Accuracy for  ResNet + Transfer Learning:  95.31996250152588 %


In [21]:
score = model_pretrained.evaluate(ds_test, steps = len(df_test), verbose = 0)

In [24]:
print('Testing Loss for ResNet + Transfer Learning: ',score[0])
print('Testing Accuracy for  ResNet + Transfer Learning: ',score[1]*100, '%')

Testing Loss for ResNet + Transfer Learning:  0.3071925938129425
Testing Accuracy for  ResNet + Transfer Learning:  87.44326829910278 %
